In [238]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [239]:
import scipy.stats as st
from functions import *
from graphics import *
from statistics_tests import *
import pandas as pd

df_final_demo = get_final_demo_df()

df_web_data_concat = get_web_data_df()

df_final_exp = get_final_exp_df()

df_all = get_df_all()

LIMPIEZA DE DATOS

In [240]:
df_all = df_all.drop("clnt_tenure_yr", axis=1)      # We don´t need this column because is derivative from clnt_tenure_mnth

QUITAMOS OUTLIERS DEL DF_ALL Y DEL QUE NO TIENE DUPLICADOS

In [241]:
df_all = outlier_slayer(df_all)

In [242]:
df_all.sort_values(by=["client_id", "visit_id", "date_time"], ascending=[True, True, False], inplace=True)

In [243]:
# df_all["time_diff"] = df_all.groupby(["client_id", "visit_id"])["date_time"].shift(1) - df_all["date_time"]

In [244]:
df_all["time_diff"] = abs(df_all.groupby(["client_id", "visit_id"])["date_time"].diff().shift(-1).fillna(pd.Timedelta(seconds=0)) ).dt.total_seconds()

In [245]:
df_inter = df_all.copy()
df_inter.drop_duplicates(subset=["visit_id", "process_step"], keep="last", inplace=True)

In [246]:
df_all["time_diff_cleaned"] = abs(df_inter.groupby(["client_id", "visit_id"])["date_time"].diff().shift(-1).fillna(pd.Timedelta(seconds=0)) ).dt.total_seconds()

In [247]:
df_all["repetition"] = df_all.duplicated(subset=["visit_id", "process_step"], keep="last").astype(int)          #0 = no, 1 = yes

In [248]:
df_all["last_step"] = df_all.groupby("visit_id")["process_step"].transform("max")

In [249]:
df_all.head()

,client_id,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,visitor_id,visit_id,process_step,date_time,variation,time_diff,time_diff_cleaned,repetition,last_step
255392,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,4,2017-04-15 13:00:34,1,20.0,20.0,0,4
255393,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,3,2017-04-15 13:00:14,1,99.0,99.0,0,4
255394,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,2,2017-04-15 12:58:35,1,32.0,32.0,0,4
255395,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,1,2017-04-15 12:58:03,1,7.0,7.0,0,4
255396,555,46.0,29.5,U,2.0,25454.66,2.0,6.0,402506806_56087378777,637149525_38041617439_716659,0,2017-04-15 12:57:56,1,0.0,0.0,0,4


In [250]:
# count = df_all[(df_all["process_step"] == 4) & 
#                 (df_all["repetition"] == 0) & 
#                 (df_all["variation"] == 1)].shape[0]

# count

In [251]:
df_all["time_diff_cleaned"][df_all["variation"] == 0].isna().sum()

23746

In [252]:
df_all["time_diff_cleaned"][df_all["variation"] == 1].isna().sum()

32859

In [253]:
df_all.shape

(220371, 17)

In [254]:
df_all[(df_all["time_diff_cleaned"] > 0) & (df_all["process_step"] == 0) ]

,client_id,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,visitor_id,visit_id,process_step,date_time,variation,time_diff,time_diff_cleaned,repetition,last_step
62812,119634,165.0,34.5,U,2.0,86417.53,2.0,5.0,820059231_33797236671,545042815_65454898838_253750,0,2017-04-13 15:49:15,1,11.0,108.0,0,3
223550,210180,58.0,39.0,U,2.0,30501.68,0.0,3.0,185274135_54378318302,225349640_17353665427_155795,0,2017-03-28 12:32:33,1,33.0,33.0,0,3
303024,281100,190.0,57.0,M,2.0,47380.01,4.0,7.0,950861310_15447304794,90791601_93065315653_131175,0,2017-04-12 23:21:47,1,12.0,293.0,0,3
308439,286957,75.0,52.5,U,2.0,32914.56,5.0,8.0,77668932_68665317144,731457989_36147011199_422595,0,2017-04-17 13:14:35,1,70.0,70.0,0,4
205401,568724,58.0,57.0,U,2.0,46326.58,3.0,6.0,872341175_14365517589,765829908_60260115119_202309,0,2017-04-09 15:05:10,1,511.0,667.0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173007,9825024,166.0,37.5,F,2.0,59812.59,2.0,5.0,38729247_28173183752,302222249_26225905626_448463,0,2017-04-15 18:49:12,1,665.0,785.0,0,3
226106,9847907,178.0,63.0,M,2.0,27629.82,0.0,3.0,160732505_33955016909,686688743_2039509682_577717,0,2017-04-01 12:19:44,1,124.0,163.0,0,3
224483,9895333,76.0,28.0,U,2.0,39979.78,3.0,6.0,998518675_65331289455,40359801_27316499959_198257,0,2017-05-15 21:41:17,1,2.0,2.0,0,4
115663,9908372,82.0,57.5,U,2.0,52673.45,0.0,3.0,66700156_72547746341,906218962_8644962559_128199,0,2017-04-12 21:02:43,1,107.0,107.0,0,3


In [255]:
df_all[df_all["client_id"] == 119634].iloc[0:100]

,client_id,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,visitor_id,visit_id,process_step,date_time,variation,time_diff,time_diff_cleaned,repetition,last_step
62816,119634,165.0,34.5,U,2.0,86417.53,2.0,5.0,820059231_33797236671,53403219_1650435963_154330,1,2017-04-13 15:46:18,1,13.0,13.0,0,1
62817,119634,165.0,34.5,U,2.0,86417.53,2.0,5.0,820059231_33797236671,53403219_1650435963_154330,0,2017-04-13 15:46:05,1,0.0,0.0,0,1
62812,119634,165.0,34.5,U,2.0,86417.53,2.0,5.0,820059231_33797236671,545042815_65454898838_253750,0,2017-04-13 15:49:15,1,11.0,108.0,0,3
62813,119634,165.0,34.5,U,2.0,86417.53,2.0,5.0,820059231_33797236671,545042815_65454898838_253750,2,2017-04-13 15:49:04,1,97.0,NaN,1,3
62814,119634,165.0,34.5,U,2.0,86417.53,2.0,5.0,820059231_33797236671,545042815_65454898838_253750,3,2017-04-13 15:47:27,1,28.0,28.0,0,3
62815,119634,165.0,34.5,U,2.0,86417.53,2.0,5.0,820059231_33797236671,545042815_65454898838_253750,2,2017-04-13 15:46:59,1,0.0,0.0,0,3
62818,119634,165.0,34.5,U,2.0,86417.53,2.0,5.0,820059231_33797236671,763070877_39147641592_532833,4,2017-04-15 16:43:47,1,20.0,20.0,0,4
62819,119634,165.0,34.5,U,2.0,86417.53,2.0,5.0,820059231_33797236671,763070877_39147641592_532833,3,2017-04-15 16:43:27,1,16.0,16.0,0,4
62820,119634,165.0,34.5,U,2.0,86417.53,2.0,5.0,820059231_33797236671,763070877_39147641592_532833,2,2017-04-15 16:43:11,1,17.0,17.0,0,4
62821,119634,165.0,34.5,U,2.0,86417.53,2.0,5.0,820059231_33797236671,763070877_39147641592_532833,1,2017-04-15 16:42:54,1,4.0,85.0,0,4


In [256]:
# df_all.to_csv("data/df_powerbi.csv")
